# POC for NCAABB Elo

In [1]:
cd ..

c:\Users\charl\Desktop\Git\elo-rating


In [4]:
from elo_runner import main
main()

Starting Runner for basketball/mens-college-basketball (2024-2024)...
Making Elo for basketball/mens-college-basketball - 2024
Starting Runner for football/college-football (2023-2023)...
Making Elo for football/college-football - 2023
Starting Runner for baseball/college-baseball (2024-2024)...
Making Elo for baseball/college-baseball - 2024
Starting Runner for hockey/mens-college-hockey (2024-2024)...
Making Elo for hockey/mens-college-hockey - 2024
Starting Runner for lacrosse/mens-college-lacrosse (2024-2024)...
Making Elo for lacrosse/mens-college-lacrosse - 2024
Starting Runner for basketball/nba (2024-2024)...
Making Elo for basketball/nba - 2024
Starting Runner for football/nfl (2023-2023)...
Making Elo for football/nfl - 2023
Starting Runner for baseball/mlb (2024-2024)...
Making Elo for baseball/mlb - 2024
Starting Runner for hockey/nhl (2024-2024)...
Making Elo for hockey/nhl - 2024
[Errno 2] No such file or directory: './data/elo/hockey/nhl/2005.parquet'
Starting Runner for

In [3]:
from src.consts import ESPNSportTypes, START_SEASONS, SEASON_GROUPS
from src.utils import find_year_for_season, create_dataframe, put_dataframe, get_dataframe
from events_runner import run_events_for_sport
from elo_runner import run_elo_for_sport
from src.event import ESPNEventsAPI
import numpy as np
import pandas as pd
import os
import datetime

## If its refresh pull whole thing
## If its upsert get the latest seasons in the fs and determine active sports that need pulling

sports = [
    ESPNSportTypes.COLLEGE_BASKETBALL,
    ESPNSportTypes.COLLEGE_FOOTBALL,
    ESPNSportTypes.COLLEGE_BASEBALL,
    ESPNSportTypes.COLLEGE_HOCKEY,
    ESPNSportTypes.COLLEGE_LACROSSE,
    ESPNSportTypes.NFL,
    ESPNSportTypes.NBA,
    ESPNSportTypes.NHL,
    ESPNSportTypes.MLB,
    ESPNSportTypes.PLL

]
import time
status_reports = {}
for sport in sports:
    start = time.time()
    try:
        run_events_for_sport(root_path='./data/events', sport=sport, espn_events_api=ESPNEventsAPI())
        run_elo_for_sport('./data/events','./data/elo',sport=sport)
        status_reports[sport] = {
            'status':True,
            'execution_time':round(time.time() - start, 2),
            'end_datetime': datetime.datetime.utcnow()
        }
    except Exception as e:
        print('FAILURE')
        print(e)
        status_reports[sport] = {
            'status':False,
            'execution_time':round(time.time() - start, 2),
            'end_datetime': datetime.datetime.utcnow()
        }
print('')
print('Events Pump Status Report')
print('-'*110)
duration = 0
for key, report in status_reports.items():
    duration = duration + report['execution_time']
    print(f"    {key}: {'PASSED' if report['status'] else 'FAILED'} -- took {report['execution_time']} sec, finished at ({report['end_datetime']}) ")
print('')
print(f'Pump took {duration} sec')
print('-'*110)



Starting Runner for basketball/mens-college-basketball (2024-2024)...
Getting Events for basketball/mens-college-basketball - 2024
    Updating data from 20240407-20240410...
Starting Runner for basketball/mens-college-basketball (2024-2024)...
Making Elo for basketball/mens-college-basketball - 2024
Starting Runner for football/college-football (2023-2023)...
Getting Events for football/college-football - 2023
    Updating data from 20240107-20240110...
Starting Runner for football/college-football (2023-2023)...
Making Elo for football/college-football - 2023
Starting Runner for baseball/college-baseball (2024-2024)...
Getting Events for baseball/college-baseball - 2024
    Updating data from 20240427-20240608...
Starting Runner for baseball/college-baseball (2024-2024)...
Making Elo for baseball/college-baseball - 2024
Starting Runner for hockey/mens-college-hockey (2024-2024)...
Getting Events for hockey/mens-college-hockey - 2024
    Updating data from 20240411-20240508...
Startin

In [ ]:
sports = [ESPNSportTypes.COLLEGE_BASKETBALL, ESPNSportTypes.COLLEGE_BASEBALL,ESPNSportTypes.COLLEGE_FOOTBALL,ESPNSportTypes.COLLEGE_HOCKEY]
SCHEMA = {
            'id':np.int64,
            'season':np.int32,
            'is_postseason': np.int8,
            'tournament_id':'Int32',
            'is_finished': np.int8,
            'neutral_site': np.int8,
            'home_team_id':np.int32,
            'home_team_score':'Int32',
            'away_team_id':np.int32,
            'away_team_score':'Int32',
        }
for sport in sports:
    for file in os.listdir(f'./data/events/{sport.value}'):
        fs_df = get_dataframe(f'./data/events/{sport.value}/{file}')
        put_dataframe(fs_df, f'./data/events/{sport.value}/{file}', SCHEMA)




In [3]:
from src.consts import ESPNSportTypes, START_SEASONS, SEASON_GROUPS
from src.event import ESPNEventsAPI
from src.sport import ESPNSport
sport = ESPNSportTypes.MLB
espn_sport_obj = ESPNSport(sport=sport, season=2024)
on_days = espn_sport_obj.ondays
espn_events_api=ESPNEventsAPI()
for i in on_days:
    res = espn_events_api.get_events_for_elo(sport,i)

In [4]:
espn_sport_obj.__dict__

{'_base_url': 'https://site.api.espn.com/apis/site/v2/sports',
 '_core_url': 'https://sports.core.api.espn.com/v2/sports',
 'sport': <ESPNSportTypes.MLB: 'baseball/mlb'>,
 'espn_core_name': 'baseball/leagues/mlb',
 'is_college_sport': False,
 'date': datetime.datetime(2024, 5, 29, 14, 33, 55, 758268),
 'season': 2024,
 'start_date': datetime.datetime(2024, 3, 20, 7, 0),
 'end_date': datetime.datetime(2024, 12, 11, 7, 59),
 'is_active': True,
 'ondays': [datetime.datetime(2024, 3, 20, 7, 0),
  datetime.datetime(2024, 3, 21, 7, 0),
  datetime.datetime(2024, 3, 22, 7, 0),
  datetime.datetime(2024, 3, 23, 7, 0),
  datetime.datetime(2024, 3, 24, 7, 0),
  datetime.datetime(2024, 3, 25, 7, 0),
  datetime.datetime(2024, 3, 26, 7, 0),
  datetime.datetime(2024, 3, 28, 7, 0),
  datetime.datetime(2024, 3, 29, 7, 0),
  datetime.datetime(2024, 3, 30, 7, 0),
  datetime.datetime(2024, 3, 31, 7, 0),
  datetime.datetime(2024, 4, 1, 7, 0),
  datetime.datetime(2024, 4, 2, 7, 0),
  datetime.datetime(2024, 

In [ ]:
espn_events_api=ESPNEventsAPI()
team_ids = []
url = 'http://sports.core.api.espn.com/v2/sports/lacrosse/leagues/mens-college-lacrosse/seasons/2008/teams'
res = espn_events_api.api_request(url+'?limit=500')
for item in res['items']:
    team_ids.append(int(item['$ref'].replace(url+'/','').split('?')[0]))
team_ids